In [1]:
from minio import Minio
from dotenv import load_dotenv
import os
import json

In [2]:
load_dotenv()

minio_client = Minio(
    endpoint=os.getenv("MINIO_ENDPOINT"),
    access_key=os.getenv("MINIO_ACCESS_KEY"), 
    secret_key=os.getenv("MINIO_SECRET_KEY"), 
    secure=False 
)
bucket_name = os.getenv("MINIO_BUCKET_NAME")

In [3]:
import re

# Lista de bairros de Fortaleza/CE
bairros_fortaleza = [
    "Aeroporto", "Aerolândia", "Alagadiço Novo", "Aldeota", "Alto da Balança",
    "Álvaro Weyne", "Amadeu Furtado", "Ancuri", "Antônio Bezerra", "Autran Nunes",
    "Barra do Ceará", "Barroso", "Bela Vista", "Benfica", "Bom Futuro",
    "Bom Jardim", "Bonsucesso", "Cais do Porto", "Cajazeiras", "Cambeba",
    "Canindezinho", "Carlito Pamplona", "Cidade 2000", "Cidade dos Funcionários", "Coaçu",
    "Cocó", "Conjunto Ceará I", "Conjunto Ceará II", "Conjunto Esperança", "Conjunto Palmeiras",
    "Couto Fernandes", "Cristo Redentor", "Curió", "Damas", "Demócrito Rocha",
    "Dendê", "Dias Macedo", "Dionísio Torres", "Dom Lustosa", "Edson Queiroz",
    "Ellery", "Engenheiro Luciano Cavalcante", "Farias Brito", "Fátima", "Floresta",
    "Genibaú", "Granja Lisboa", "Granja Portugal", "Guajeru", "Guararapes",
    "Henrique Jorge", "Itaoca", "Itaperi", "Jacarecanga", "Jangurussu",
    "Jardim América", "Jardim Cearense", "Jardim das Oliveiras", "Jardim Guanabara", "Jardim Iracema",
    "João XXIII", "Joaquim Távora", "Jóquei Clube", "José Bonifácio", "José de Alencar",
    "Lagoa Redonda", "Manuel Dias Branco", "Manoel Sátiro", "Maraponga", "Messejana",
    "Meireles", "Mondubim", "Monte Castelo", "Montese", "Moura Brasil",
    "Mucuripe", "Panamericano", "Papicu", "Parangaba", "Parque Araxá",
    "Parque Dois Irmãos", "Parque Iracema", "Parque Manibura", "Parquelândia", "Parreão",
    "Passaré", "Paupina", "Pedras", "Pici", "Pirambu",
    "Planalto Ayrton Senna", "Praia de Iracema", "Praia do Futuro I", "Praia do Futuro II", "Prefeito José Walter",
    "Presidente Kennedy", "Presidente Vargas", "Quintino Cunha", "Rodolfo Teófilo", "Sabiaguaba",
    "Salinas", "Santa Rosa", "São Bento", "São Gerardo", "São João do Tauape",
    "São José", "Sapiranga", "Serrinha", "Siqueira", "Varjota",
    "Vicente Pinzon", "Vila Ellery", "Vila Peri", "Vila União", "Vila Velha"
]


mensagens_com_bairros = []

In [4]:
import unicodedata
def normalizar_texto(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()

In [5]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]

In [8]:
bairros_normalizados

['aeroporto',
 'aerolandia',
 'alagadico novo',
 'aldeota',
 'alto da balanca',
 'alvaro weyne',
 'amadeu furtado',
 'ancuri',
 'antonio bezerra',
 'autran nunes',
 'barra do ceara',
 'barroso',
 'bela vista',
 'benfica',
 'bom futuro',
 'bom jardim',
 'bonsucesso',
 'cais do porto',
 'cajazeiras',
 'cambeba',
 'canindezinho',
 'carlito pamplona',
 'cidade 2000',
 'cidade dos funcionarios',
 'coacu',
 'coco',
 'conjunto ceara i',
 'conjunto ceara ii',
 'conjunto esperanca',
 'conjunto palmeiras',
 'couto fernandes',
 'cristo redentor',
 'curio',
 'damas',
 'democrito rocha',
 'dende',
 'dias macedo',
 'dionisio torres',
 'dom lustosa',
 'edson queiroz',
 'ellery',
 'engenheiro luciano cavalcante',
 'farias brito',
 'fatima',
 'floresta',
 'genibau',
 'granja lisboa',
 'granja portugal',
 'guajeru',
 'guararapes',
 'henrique jorge',
 'itaoca',
 'itaperi',
 'jacarecanga',
 'jangurussu',
 'jardim america',
 'jardim cearense',
 'jardim das oliveiras',
 'jardim guanabara',
 'jardim iracema'

In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [9]:
dados = []
list_objects = minio_client.list_objects(bucket_name, prefix='Portalnoticiasceara', recursive=True)
# Processar cada objeto encontrado
for obj in list_objects:
    if obj.object_name.endswith('.json'):  # Verificar se é um arquivo JSON
        print(f"Lendo o arquivo: {obj.object_name}")
        try:
            # Baixar o objeto
            response = minio_client.get_object(bucket_name=bucket_name, object_name=obj.object_name)
            
            # Ler o conteúdo
            data = response.read().decode('utf-8')
            
            # Parsear JSON
            json_data = json.loads(data)
            
            # Verificar se é uma lista
            if isinstance(json_data, list):
                for item in json_data:
                    # Extrair informações relevantes
                    message = item.get('message', '')
                    message_id = item.get('id', None)
                    date = item.get('date', None)
                    media_path = item.get('media_path', None)
                    
                    if message:
                        # Normalizar a mensagem
                        message_normalizada = normalizar_texto(message)
                        
                        # Identificar bairros mencionados
                        bairros_mencionados = [
                            bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                            if bairro_normalizado in message_normalizada
                        ]
                        
                        # Se algum bairro foi identificado, salvar os dados
                        if bairros_mencionados:
                            for bairro in bairros_mencionados:
                                dados.append({
                                    "id": message_id,
                                    "mensagem": message,
                                    "bairro": bairro,
                                    "data": date,
                                    "media_path": media_path
                                })
            
            # Fechar o response
            response.close()
            response.release_conn()
            
            with open('./silver_metadata.json', 'w', encoding='utf-8') as json_file:
                json.dump(dados, json_file, ensure_ascii=False, indent=4)

        except Exception as e:
            print(f"Erro ao processar {obj.object_name}: {e}")

MaxRetryError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /raw-portaldenoticias?location= (Caused by ProtocolError('Connection aborted.', BadStatusLine('ÿ\x00\x00\x00\x00\x00\x00\x00\x01\x7f\n')))

In [21]:
df = pd.DataFrame(dados)

In [22]:
df.tail()

,id,mensagem,bairro,data,media_path
453,5954,"**ATENÇÃO 🚨** HOMICÍDIO REGISTRADO A POUCOS MINUTOS PRÓXIMO DA LAGOA DA ZEZA,BAIRRO JARDIM DAS OLIVEIRAS (TDN) EM FORTALEZA.\n\nSMD",Jardim das Oliveiras,2024-12-17T19:59:29+00:00,None
454,5958,"Homicídio registrado a poucos minutos próximo a lagoa da ZEZA (TDN) bairro jardim das oliveiras em fortaleza.\n\nVítima do sexo masculino,sem identificação até o momento, estaria em uma motocicleta quando foi alvejado por vários disparos de arma de fogo,vindo a óbito no local.",Jardim das Oliveiras,2024-12-17T20:20:15+00:00,Portalnoticiasceara/2024-12-17/media/photo_5958.jpg.zstd
455,5962,Atenção 🚨rajada de tiros a poucos instante no planalto Ayrton Senna em fortaleza. \nMovimentação policial nesse momento na região.\n\nAguardando mas detalhes.,Planalto Ayrton Senna,2024-12-17T21:13:42+00:00,None
456,5963,ATENÇÃO 🚨 BALA NESTE EXATO MOMENTO PRÓXIMO A RUA DIAMANTE NO BARROSO 2 EM FORTALEZA. \n\nAGUARDANDO MAS DETALHES.,Barroso,2024-12-17T21:27:57+00:00,None
457,5987,"Homicídio agora nesse exato momento no Conjunto Palmeiras, na rua Oscarito\nPerto da padaria do Moreira\nO MESMO NÃO TINHA NENHUM ENVOLVIMENTO, MAIS NÃO SE SABE O MOTIVO, \n\nSMD AMD.",Conjunto Palmeiras,2024-12-18T09:39:21+00:00,Portalnoticiasceara/2024-12-18/media/photo_5987.jpg.zstd


In [31]:
df[df['bairro'] == 'Barra do Ceará'].sort_values(by='data', ascending=False)

,id,mensagem,bairro,data,media_path
435,5852,"Barra do Ceará, fortaleza.\n🚨🚨",Barra do Ceará,2024-12-16T10:35:09+00:00,Portalnoticiasceara/2024-12-16/media/photo_5852.jpg.zstd
398,5724,Atualização 🚨\n\nTriplo homicídio na colônia (GDE)\nBarra do Ceará.\n\nhttps://t.me/Portalnoticiasceara/5719,Barra do Ceará,2024-12-14T07:50:04+00:00,Portalnoticiasceara/2024-12-14/media/photo_5724.jpg.zstd
396,5719,"**Triplo homicídio registrado na madrugada deste sábado na Colônia, Barra do Ceará**\n\nNa madrugada deste sábado (14/12), por volta das 4h33, um triplo homicídio foi registrado na comunidade da Colônia (GDE), na Barra do Ceará, Fortaleza. De acordo com informações recebidas pelo Portal Notícias CE, indivíduos chegaram em um veículo na rua Grito de Alerta, se passando por policiais civis.\n\nAo avistarem três pessoas, ordenaram que colocassem as mãos na cabeça, simulando uma abordagem policial. Logo em seguida, efetuaram diversos disparos de arma de fogo contra as vítimas, que não tiveram nenhuma chance de defesa. Testemunhas relataram ter ouvido cerca de 20 tiros ou mais.\n\nEquipes da polícia e do rabecão foram acionadas e estiveram no local para os procedimentos iniciais e a remoção dos corpos. Até o momento, não há informações sobre a identidade ou o sexo das vítimas, tampouco se possuíam ligação com atividades criminosas.\n\nO Portal Notícias CE continuará acompanhando o caso e trará novas atualizações assim que disponíveis.",Barra do Ceará,2024-12-14T07:00:44+00:00,None
395,5713,**ATENÇÃO 🚨** CHUVA DE BALAS NESTE EXATO MOMENTO NA COMUNIDADE DA COLÔNIA (GDE) BARRA DO CEARÁ EM FORTALEZA.\n\nINTENSA TROCA DE TIROS ENTRE FACÇÕES RIVAIS ( CV X GDE ).\n\nSMD,Barra do Ceará,2024-12-14T04:33:22+00:00,None
348,5481,**ATENÇÃO 🚨** MUITA BALA NESTE EXATO MOMENTO NA COMUNIDADE DA COLÔNIA (GDE) BAIRRO BARRA DO CEARÁ EM FORTALEZA.\n\nSMD,Barra do Ceará,2024-12-11T17:38:56+00:00,None
341,5465,"**Polícia Civil captura alvos envolvidos em crimes na Região Norte do Estado\n\n**Um intenso trabalho de combate aos grupos criminosos no Estado realizado pela Polícia Civil do Estado do Ceará (PCCE) resultou no cumprimento de mandados judiciais de prisão preventiva, regressão cautelar e de sentença condenatória na Capital e na Região Norte. As ações policiais aconteceram nessa segunda-feira (9) e nesta terça-feira (10).\nUm dos alvos, um homem de 21 anos, é investigado pelo crime de homicídio qualificado. Além desse crime, o suspeito também é investigado por outros homicídios nas cidades de Amontada – Área Integrada de Segurança (AIS 17) do Estado, Miraíma (AIS 17) e nos bairros Pirambu (AIS 8) e Barra do Ceará (AIS 8) da Capital. A captura do suspeito que tinha um mandado de prisão preventiva em aberto, aconteceu no bairro Bom Jardim (AIS 2) de Fortaleza.\nUm outro suspeito, de 27 anos, é investigado pelos crimes de ameaça e descumprimento de medida protetiva de urgência. Ele foi preso em um imóvel no município de Itapipoca (AIS 17) e conduzido para a Delegacia Regional de Itapipoca, onde foi cumprido o mandado de regressão cautelar.\nO terceiro alvo, capturado em Sobral (AIS 14), possuía um mandado de prisão por sentença condenatória em aberto pelo crime de lesão corporal dolosa seguida de morte. Ele já responde pelos crimes de tentativa de homicídio, tráfico de drogas, roubo à pessoa e porte ilegal de arma de fogo.\nOs três indivíduos foram conduzidos para as unidades policiais, onde as ordens judiciais foram cumpridas. Agora, os homens estão à disposição do Poder Judiciário.",Barra do Ceará,2024-12-11T14:02:36+00:00,None
314,5384,"**Homem é preso armado e com drogas no Campo do Grêmio, em Fortaleza**\n\nNa tarde deste domingo, 8 de dezembro, um indivíduo conhecido pelo vulgo ""Playboy"" foi preso no Campo do Grêmio (GDE) barra do Ceará, em Fortaleza. Durante a abordagem, foram encontrados com ele uma arma de fogo e uma quantidade de drogas, que não foi especificada.",Barra do Ceará,2024-12-09T19:16:53+00:00,Portalnoticiasceara/2024-12-09/media/phot

In [28]:
df['bairro'].value_counts().head(20)

bairro
Barra do Ceará           42
Pici                     26
Messejana                25
Jardim das Oliveiras     22
Carlito Pamplona         22
Pirambu                  20
Vila Velha               17
Barroso                  17
Jangurussu               16
Edson Queiroz            15
Pedras                   11
Conjunto Palmeiras       11
Bom Jardim               10
São José                 10
Paupina                   8
Mondubim                  8
Planalto Ayrton Senna     8
Quintino Cunha            8
Álvaro Weyne              8
Cocó                      8
Name: count, dtype: int64

In [24]:
import zstandard
decompressor = zstandard.ZstdDecompressor()

In [25]:
def download_and_decompress(path, bucket_name):
    # Baixar o arquivo do MinIO
    result = minio_client.get_object(bucket_name, path)
    
    # Ler os dados comprimidos
    compressed_data = result.read()
    
    # Descomprimir os dados
    decompressor = zstandard.ZstdDecompressor()
    
    try:
        # Usando BytesIO para garantir que os dados sejam tratados como fluxo
        decompressed_data = decompressor.decompress(compressed_data)
        return decompressed_data
    except zstandard.ZstdError as e:
        print(f"Erro ao descomprimir o arquivo {path}: {e}")
        return None

In [26]:
paths = df[df['id'] == 5777]['media_path']  # Carregar os paths do dataframe

for path in paths:
    print(f"Processando {path}...")
    decompressed_data = download_and_decompress(path, bucket_name)
    
    if decompressed_data:
        print(f"Arquivo {path} descomprimido com sucesso.")
        # Aqui você pode salvar ou processar os dados descomprimidos
    else:
        print(f"Falha ao descomprimir o arquivo {path}.")

Processando Portalnoticiasceara/2024-12-15/media/photo_5777.jpg.zstd...
Erro ao descomprimir o arquivo Portalnoticiasceara/2024-12-15/media/photo_5777.jpg.zstd: could not determine content size in frame header
Falha ao descomprimir o arquivo Portalnoticiasceara/2024-12-15/media/photo_5777.jpg.zstd.
